## IMDb Datasets Analysis

* **Collaborators:** Mingrui Zhang
* **ID:** 20985422

In [8]:
# Java, Spark install
!apt-get update -qq > /dev/null
!apt-get install openjdk-8-jdk-headless -qq > /dev/null
!wget -q https://www.student.cs.uwaterloo.ca/~cs451/content/cs431/spark-2.4.8-bin-hadoop2.7.tgz
!tar xf spark-2.4.8-bin-hadoop2.7.tgz
!pip install -q findspark

# Download IMDb Datasets and unzip it as tsv files
!wget -q https://datasets.imdbws.com/name.basics.tsv.gz
!wget -q https://datasets.imdbws.com/title.akas.tsv.gz
!wget -q https://datasets.imdbws.com/title.basics.tsv.gz
!wget -q https://datasets.imdbws.com/title.crew.tsv.gz
!wget -q https://datasets.imdbws.com/title.episode.tsv.gz
!wget -q https://datasets.imdbws.com/title.principals.tsv.gz
!wget -q https://datasets.imdbws.com/title.ratings.tsv.gz

# nameBasics
!gzip -d name.basics.tsv.gz
!mv name.basics.tsv nameBasics.tsv

# titleAkas
!gzip -d title.akas.tsv.gz
!mv title.akas.tsv titleAkas.tsv

# titleBasics
!gzip -d title.basics.tsv.gz
!mv title.basics.tsv titleBasics.tsv

# titleCrew
!gzip -d title.crew.tsv.gz
!mv title.crew.tsv titleCrew.tsv

# titleEpisode
!gzip -d title.episode.tsv.gz
!mv title.episode.tsv titleEpisode.tsv

# titlePrincipals
!gzip -d title.principals.tsv.gz
!mv title.principals.tsv titlePrincipals.tsv

# titleRatings
!gzip -d title.ratings.tsv.gz
!mv title.ratings.tsv titleRatings.tsv

In [9]:
import os
os.environ["JAVA_HOME"] = "/usr/lib/jvm/java-8-openjdk-amd64"
os.environ["SPARK_HOME"] = "/content/spark-2.4.8-bin-hadoop2.7"

import findspark
findspark.init()

from pyspark.sql import SparkSession
import random

spark = SparkSession.builder.appName("YourTest").master("local[2]").config('spark.ui.port', random.randrange(4000,5000)).getOrCreate()

In [12]:
nameBasicRaw = spark.read.option("header","true").option("sep", "\t") \
        .option("multiLine", "true") \
        .option("quote","\"") \
        .option("escape","\"") \
        .option("ignoreTrailingWhiteSpace", True) \
        .csv("nameBasics.tsv")

nameBasicRaw.show()

nameBasicRaw.dtypes

+---------+-------------------+---------+---------+--------------------+--------------------+
|   nconst|        primaryName|birthYear|deathYear|   primaryProfession|      knownForTitles|
+---------+-------------------+---------+---------+--------------------+--------------------+
|nm0000001|       Fred Astaire|     1899|     1987|soundtrack,actor,...|tt0072308,tt00531...|
|nm0000002|      Lauren Bacall|     1924|     2014|  actress,soundtrack|tt0117057,tt00383...|
|nm0000003|    Brigitte Bardot|     1934|       \N|actress,soundtrac...|tt0056404,tt00573...|
|nm0000004|       John Belushi|     1949|     1982|actor,soundtrack,...|tt0078723,tt00804...|
|nm0000005|     Ingmar Bergman|     1918|     2007|writer,director,a...|tt0050976,tt00839...|
|nm0000006|     Ingrid Bergman|     1915|     1982|actress,soundtrac...|tt0077711,tt00381...|
|nm0000007|    Humphrey Bogart|     1899|     1957|actor,soundtrack,...|tt0037382,tt00345...|
|nm0000008|      Marlon Brando|     1924|     2004|actor,sou

[('nconst', 'string'),
 ('primaryName', 'string'),
 ('birthYear', 'string'),
 ('deathYear', 'string'),
 ('primaryProfession', 'string'),
 ('knownForTitles', 'string')]